# 24083010100 - REGRESI LINEAR BERGANDA

# **PREPROCESSING DATA**

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import jarque_bera, norm

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
plt.rcParams["figure.dpi"] = 100

# =====================================================
# 1. LOAD DATA DARI KAGGLE
# =====================================================
train_df = pd.read_csv("/kaggle/input/statistika-regresi-sains-data-upnvjt-regresi/train_dataset.csv")
test_df  = pd.read_csv("/kaggle/input/statistika-regresi-sains-data-upnvjt-regresi/test_dataset_no_y.csv")

print("Train shape:", train_df.shape)
print("Test  shape:", test_df.shape)

TARGET   = "y"
cat_cols = ["x2", "x3", "x5", "x6"]
num_cols = ["x1", "x4", "x7", "x8", "x9", "x10"]

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
print("\n[2] CEK MISSING VALUE – TRAIN")
print(train_df.isnull().sum())

print("\n[2] CEK MISSING VALUE – TEST")
print(test_df.isnull().sum())


In [ ]:
print("\n[3] Statistik Deskriptif Awal (y)")
print(train_df["y"].describe())

y_raw = train_df["y"].dropna()

plt.figure(figsize=(12, 4))

# Histogram y + kurva normal
plt.subplot(1, 2, 1)
ax = sns.histplot(y_raw, kde=False, stat="density", bins=30)
mean_y = y_raw.mean()
std_y  = y_raw.std()
x_vals = np.linspace(y_raw.min(), y_raw.max(), 200)
pdf    = norm.pdf(x_vals, loc=mean_y, scale=std_y)
plt.plot(x_vals, pdf, color="red", label="Normal PDF")
plt.title("Histogram y (awal) + kurva normal")
plt.legend()

# Boxplot y
plt.subplot(1, 2, 2)
sns.boxplot(x=y_raw)
plt.title("Boxplot y (awal)")

plt.tight_layout()
plt.show()


# **DATA CLEANING**

In [ ]:
# Drop baris yang y-nya hilang
train_df = train_df.dropna(subset=[TARGET]).copy()
print(f"\n[4] Setelah drop missing y: {train_df.shape[0]} observasi")

# Simpan statistik imputasi untuk test
train_stats = {}

# Imputasi kategori dengan modus
for col in cat_cols:
    if col in train_df.columns:
        mode_val = train_df[col].mode()
        if len(mode_val) > 0:
            mode_val = mode_val[0]
            train_stats[f"{col}_mode"] = mode_val
            train_df[col] = train_df[col].fillna(mode_val)

# Imputasi numerik dengan median
for col in num_cols:
    if col in train_df.columns:
        median_val = train_df[col].median()
        train_stats[f"{col}_median"] = median_val
        train_df[col] = train_df[col].fillna(median_val)

print("\n[4] Imputasi kategori (modus) dan numerik (median) selesai")

# Trimming outlier y dengan IQR
Q1 = train_df[TARGET].quantile(0.25)
Q3 = train_df[TARGET].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outlier_mask = (train_df[TARGET] < lower) | (train_df[TARGET] > upper)
n_outliers = outlier_mask.sum()

print(f"\n[4] Deteksi Outlier (IQR 1.5x pada y)")
print(f"    Q1: {Q1:.3f}, Q3: {Q3:.3f}, IQR: {IQR:.3f}")
print(f"    Batas: [{lower:.3f}, {upper:.3f}]")
print(f"    Jumlah outlier: {n_outliers} ({n_outliers/len(train_df)*100:.2f}%)")

train_df = train_df[~outlier_mask].copy()
print(f"    Setelah trimming y: {train_df.shape[0]} observasi")

In [ ]:
y_trim = train_df[TARGET]

plt.figure(figsize=(12, 4))

# Histogram y_trim + kurva normal
plt.subplot(1, 2, 1)
ax = sns.histplot(y_trim, kde=False, stat="density", bins=30)
mean_y_t = y_trim.mean()
std_y_t  = y_trim.std()
x_vals   = np.linspace(y_trim.min(), y_trim.max(), 200)
pdf_t    = norm.pdf(x_vals, loc=mean_y_t, scale=std_y_t)
plt.plot(x_vals, pdf_t, color="red", label="Normal PDF")
plt.title("Histogram y (setelah trimming) + kurva normal")
plt.legend()

# Boxplot y_trim
plt.subplot(1, 2, 2)
sns.boxplot(x=y_trim)
plt.title("Boxplot y (setelah trimming)")

plt.tight_layout()
plt.show()


# MODEL OLS ROBUST

In [ ]:
feature_cols = ["x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10"]

y_f = train_df[TARGET]
X_full = train_df[feature_cols].copy()

# One-hot encoding variabel kategorik
X_full_encoded = pd.get_dummies(X_full, columns=cat_cols, drop_first=True)

# Train–validation split
X_train, X_val, y_train, y_val = train_test_split(
    X_full_encoded, y_full, test_size=0.2, random_state=42
)

# Samakan kolom antara train dan val
X_val = X_val.reindex(columns=X_train.columns, fill_value=0)

print(f"\nTrain-Validation Split (80-20)")
print(f"    Train      : {X_train.shape[0]} observasi")
print(f"    Validation : {X_val.shape[0]} observasi")


In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, r2_score

# Ambil hanya variabel signifikan dari data yang sudah dipreproses & di-split
sig_cols = ['x1', 'x8', 'x9', 'x10']  # variabel signifikan dari output OLS robust

X_train_sig = X_train[sig_cols]
X_val_sig   = X_val[sig_cols]

# Tambahkan konstanta
X_train_sig_c = sm.add_constant(X_train_sig)
X_val_sig_c   = sm.add_constant(X_val_sig)

# Estimasi OLS dengan standard error robust HC3
model_sig = sm.OLS(y_train, X_train_sig_c).fit(cov_type='HC3')

print("="*70)
print("RINGKASAN MODEL OLS ROBUST (VARIABEL SIGNIFIKAN SAJA)")
print("="*70)
print(model_sig.summary())

# Evaluasi performa
y_train_pred = model_sig.predict(X_train_sig_c)
y_val_pred   = model_sig.predict(X_val_sig_c)

mse_train_sig = mean_squared_error(y_train, y_train_pred)
mse_val_sig   = mean_squared_error(y_val, y_val_pred)
r2_train_sig  = r2_score(y_train, y_train_pred)

print("\nMSE Train (sig only):", mse_train_sig)
print("MSE Val   (sig only):", mse_val_sig)
print("R2 Train  (sig only):", r2_train_sig)


In [ ]:
model_final = sm.OLS(y_train, X_train_sig_c).fit(cov_type='HC3')
print(model_final.summary())

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import jarque_bera

# Asumsi model_sig sudah dibuat seperti sebelumnya (x1, x8, x9, x10)
resid = model_sig.resid
X_sig_train_c = model_sig.model.exog   # desain matrix dengan konstanta


# UJI NORMALITAS

In [ ]:
jb_stat, jb_pvalue, skew, kurt = jarque_bera(resid)

print("=== UJI NORMALITAS (JARQUE-BERA) ===")
print(f"JB Statistic : {jb_stat:.4f}")
print(f"p-value      : {jb_pvalue:.6f}")
print(f"Skewness     : {skew:.4f}")
print(f"Kurtosis     : {kurt:.4f}")

# UJI HETEROSKEDASTISITAS

In [ ]:
bp_stat, bp_pvalue, _, _ = het_breuschpagan(resid, X_sig_train_c)

print("\n=== UJI HETEROSKEDASTISITAS (BREUSCH-PAGAN) ===")
print(f"LM Statistic : {bp_stat:.4f}")
print(f"p-value      : {bp_pvalue:.6f}")

# UJI AUTOKORELASI

In [ ]:
dw_stat = sm.stats.durbin_watson(resid)

print("\n=== UJI AUTOKORELASI (DURBIN-WATSON) ===")
print(f"DW Statistic : {dw_stat:.4f}")

# UJI MULTIKOLINEARITAS

In [ ]:
import pandas as pd

X_for_vif = pd.DataFrame(X_sig_train_c, columns=model_sig.model.exog_names)
X_for_vif = X_for_vif.drop(columns=['const'])

vif_data = []
for i, col in enumerate(X_for_vif.columns):
    vif_data.append((col, variance_inflation_factor(X_for_vif.values, i)))

vif_df = pd.DataFrame(vif_data, columns=['Variable','VIF'])
print("\n=== VIF (MULTIKOLINEARITAS) ===")
print(vif_df)

In [ ]:
# =====================================================
# VISUALISASI RESIDUAL MODEL OLS ROBUST HC3 (SIG ONLY)
# =====================================================
resid_hc3  = model_sig.resid
fitted_hc3 = model_sig.fittedvalues

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1) Residual vs Fitted
axes[0,0].scatter(fitted_hc3, resid_hc3, alpha=0.5)
axes[0,0].axhline(0, color='red', linestyle='--')
axes[0,0].set_xlabel("Fitted values")
axes[0,0].set_ylabel("Residuals")
axes[0,0].set_title("Residual vs Fitted (OLS HC3, sig only)")

# 2) Histogram residual + KDE
sns.histplot(resid_hc3, kde=True, ax=axes[0,1])
axes[0,1].set_xlabel("Residuals")
axes[0,1].set_title("Histogram Residual (OLS HC3, sig only)")

# 3) QQ Plot
sm.qqplot(resid_hc3, line='45', ax=axes[1,0])
axes[1,0].set_title("QQ Plot Residual (OLS HC3, sig only)")

# 4) Scale–Location (Spread vs Fitted)
axes[1,1].scatter(fitted_hc3, np.sqrt(np.abs(resid_hc3)), alpha=0.5)
axes[1,1].set_xlabel("Fitted values")
axes[1,1].set_ylabel("sqrt(|Residuals|)")
axes[1,1].set_title("Scale–Location (OLS HC3, sig only)")

plt.tight_layout()
plt.show()

In [ ]:
# 6. Prediksi test & simpan submission
y_pred_test = model_final.predict(X_test_sig)

submission = pd.DataFrame({
    "ID": np.arange(1, len(y_test_pred) + 1),
    "y": y_test_pred
})

submission.to_csv("submission_olshc3_sig.csv", index=False)
print("submission_olshc3_sig.csv dibuat.")

In [ ]:
submission.head(10)